In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import upload_data, pull_model, pull_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get the latest 2 days of transformed ethereum data

In [2]:
# we'll later also need to update the day's before yesterday data for tmw (tmw_avg_high_close, tmw_percent_increase)
eth_last_2_days = pull_data('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)[-2:]
eth_today = eth_last_2_days[-1:]

eth_last_2_days

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.34s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
3088,2024-05-18,3097.989661,3144.431575,3088.35738,3120.549378,8.357435e+09,2.683836e+06,3.752063e+11,0.0,-0.0063,...,14.0,11.0,10.0,8.0,3.0,50.0,36.0,25.0,7.0,3.0
3089,2024-05-19,3127.490439,3130.454914,3057.74128,3071.324418,6.983634e+09,2.260053e+06,3.685981e+11,NaN,NaN,...,14.0,11.0,10.0,8.0,3.0,50.0,36.0,25.0,7.0,3.0


### Get model from our Hopsworks model registry

In [3]:
MODEL_NAME = 'catboost_eth_returns' # this one predicts tmw_1_0_percent_increase_binary

# ***** later implement code that checks the latest model version and pulls that one
# def pull_model(model_name, model_version):
model = pull_model(MODEL_NAME, 1)[0]

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.


### Define predictors, the input data points, and get a prediction for tmw_1_0_percent_increase_binary

In [4]:
predictors_96 = ['rsi_2', 'volume_rsi_2', 'spread_rsi_2', 'last_2_day_80th_pct_volume_change_count', 'last_2_day_60th_to_70th_pct_spread_count', 'last_2_day_40th_pct_spread_count', 'rsi_5', 'volume_rsi_5', 'spread_rsi_5', 'last_5_day_60th_pct_price_change_count', 'last_5_day_70th_pct_price_change_count', 'rsi_10', 'volume_rsi_10', 'spread_rsi_10', 'last_10_day_40th_to_50th_pct_price_change_count', 'last_10_day_60th_pct_price_change_count', 'last_10_day_70th_to_80th_pct_volume_change_count', 'last_10_day_80th_to_90th_pct_spread_count', 'last_10_day_40th_pct_spread_count', 'last_10_day_50th_pct_spread_count', 'last_10_day_70th_pct_spread_count', 'last_10_day_80th_pct_spread_count', 'rsi_25', 'volume_rsi_25', 'spread_rsi_25', 'last_25_day_40th_to_50th_pct_price_change_count', 'last_25_day_70th_to_80th_pct_price_change_count', 'last_25_day_80th_to_90th_pct_price_change_count', 'last_25_day_40th_pct_price_change_count', 'last_25_day_50th_pct_price_change_count', 'last_25_day_80th_pct_price_change_count', 'last_25_day_40th_to_50th_pct_volume_change_count', 'last_25_day_50th_to_60th_pct_volume_change_count', 'last_25_day_60th_to_70th_pct_volume_change_count', 'last_25_day_70th_to_80th_pct_volume_change_count', 'last_25_day_80th_to_90th_pct_volume_change_count', 'last_25_day_50th_pct_volume_change_count', 'last_25_day_60th_pct_volume_change_count', 'last_25_day_70th_pct_volume_change_count', 'last_25_day_80th_pct_volume_change_count', 'last_25_day_40th_to_50th_pct_spread_count', 'last_25_day_50th_to_60th_pct_spread_count', 'last_25_day_60th_to_70th_pct_spread_count', 'last_25_day_70th_to_80th_pct_spread_count', 'last_25_day_80th_to_90th_pct_spread_count', 'last_25_day_40th_pct_spread_count', 'last_25_day_60th_pct_spread_count', 'last_25_day_70th_pct_spread_count', 'last_25_day_90th_pct_spread_count', 'rsi_50', 'last_50_day_50th_to_60th_pct_price_change_count', 'last_50_day_60th_to_70th_pct_price_change_count', 'last_50_day_70th_to_80th_pct_price_change_count', 'last_50_day_40th_pct_price_change_count', 'last_50_day_50th_pct_price_change_count', 'last_50_day_70th_pct_price_change_count', 'last_50_day_40th_to_50th_pct_volume_change_count', 'last_50_day_50th_to_60th_pct_volume_change_count', 'last_50_day_50th_pct_volume_change_count', 'last_50_day_80th_pct_volume_change_count', 'last_50_day_40th_to_50th_pct_spread_count', 'last_50_day_50th_to_60th_pct_spread_count', 'last_50_day_60th_to_70th_pct_spread_count', 'last_50_day_70th_to_80th_pct_spread_count', 'last_50_day_80th_to_90th_pct_spread_count', 'last_50_day_40th_pct_spread_count', 'last_50_day_50th_pct_spread_count', 'last_50_day_70th_pct_spread_count', 'last_50_day_90th_pct_spread_count', 'rsi_100', 'last_100_day_60th_to_70th_pct_price_change_count', 'last_100_day_80th_to_90th_pct_price_change_count', 'last_100_day_90th_to_95th_pct_price_change_count', 'last_100_day_40th_pct_price_change_count', 'last_100_day_50th_pct_price_change_count', 'last_100_day_60th_pct_price_change_count', 'last_100_day_80th_pct_price_change_count', 'last_100_day_40th_to_50th_pct_volume_change_count', 'last_100_day_50th_to_60th_pct_volume_change_count', 'last_100_day_60th_to_70th_pct_volume_change_count', 'last_100_day_70th_to_80th_pct_volume_change_count', 'last_100_day_80th_to_90th_pct_volume_change_count', 'last_100_day_40th_pct_volume_change_count', 'last_100_day_60th_pct_volume_change_count', 'last_100_day_80th_pct_volume_change_count', 'last_100_day_90th_pct_volume_change_count', 'last_100_day_40th_to_50th_pct_spread_count', 'last_100_day_50th_to_60th_pct_spread_count', 'last_100_day_60th_to_70th_pct_spread_count', 'last_100_day_70th_to_80th_pct_spread_count', 'last_100_day_90th_to_95th_pct_spread_count', 'last_100_day_40th_pct_spread_count', 'last_100_day_50th_pct_spread_count', 'last_100_day_60th_pct_spread_count', 'last_100_day_80th_pct_spread_count', 'last_100_day_90th_pct_spread_count']
response_var = 'tmw_1_0_percent_increase_binary'

x0 = eth_today[predictors_96]
x0

,rsi_2,volume_rsi_2,spread_rsi_2,last_2_day_80th_pct_volume_change_count,last_2_day_60th_to_70th_pct_spread_count,last_2_day_40th_pct_spread_count,rsi_5,volume_rsi_5,spread_rsi_5,last_5_day_60th_pct_price_change_count,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
3089,48.942,17.943208,36.925916,0.0,0.0,0.0,57.27565,37.215467,43.583758,2.0,...,14.0,11.0,10.0,8.0,3.0,50.0,36.0,25.0,7.0,3.0


In [5]:
prediction = round(model.predict_proba(x0)[:, 1][0], 6)
prediction

0.364675

### Update our Hopsworks predictions data with this new prediction

In [6]:
previous_predictions = pull_data('eth_ohlc_predictions', 2, 'eth_ohlc_predictions', 2)
previous_predictions

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (0.82s) 


,date,high,close,tmw_percent_increase_to_avg_high_low,tmw_1_0_percent_increase_binary,predicted_probability
0,2018-08-11,334.260000,322.110000,0.006116,0.0,0.422378
1,2018-08-12,328.590000,319.570000,-0.045514,0.0,0.248157
2,2018-08-13,323.550000,286.500000,-0.013438,0.0,0.362669
3,2018-08-14,286.370000,278.930000,0.050353,1.0,0.645693
4,2018-08-15,303.590000,282.360000,0.038621,1.0,0.408482
...,...,...,...,...,...,...
2085,2024-05-15,3040.591286,3035.758955,-0.016662,0.0,0.396109
2086,2024-05-16,3026.768367,2943.585128,0.055185,1.0,0.386518
2087,2024-05-17,3116.054991,3095.996844,0.011787,1.0,0.413199
2088,2024-05-18,3144.431575,3120.549378,-0.004991,0.0,0.372555


In [7]:
cols = ['date', 'high', 'close', 'tmw_percent_increase_to_avg_high_low', 'tmw_1_0_percent_increase_binary']
eth_today_updated = eth_today[cols]
eth_today_updated['predicted_probability'] = prediction

eth_today_updated

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,high,close,tmw_percent_increase_to_avg_high_low,tmw_1_0_percent_increase_binary,predicted_probability
3086,2024-05-16,3026.768367,2943.585128,NaN,NaN,0.386518


In [8]:
# Check if the date from eth_yesterday is not in previous_predictions
if eth_today_updated['date'].iloc[0] not in previous_predictions['date'].values:

    # Append eth_yesterday to previous_predictions
    predictions_updated = pd.concat([previous_predictions, eth_today_updated], ignore_index=True)

    # Update the last row of previous_predictions with values from eth_last_2_days
    predictions_updated.at[predictions_updated.index[-2], 'tmw_percent_increase_to_avg_high_low'] = eth_last_2_days['tmw_percent_increase_to_avg_high_low'].iloc[-2]
    predictions_updated.at[predictions_updated.index[-2], 'tmw_1_0_percent_increase_binary'] = eth_last_2_days['tmw_1_0_percent_increase_binary'].iloc[-2]
else:
    predictions_updated = previous_predictions
    print('eth_yesterday was already in previous_predictions')

### Put back into Hopsworks

In [10]:
predictions_updated

,date,high,close,tmw_percent_increase_to_avg_high_low,tmw_1_0_percent_increase_binary,predicted_probability
0,2018-08-11,334.260000,322.110000,0.006116,0.0,0.422378
1,2018-08-12,328.590000,319.570000,-0.045514,0.0,0.248157
2,2018-08-13,323.550000,286.500000,-0.013438,0.0,0.362669
3,2018-08-14,286.370000,278.930000,0.050353,1.0,0.645693
4,2018-08-15,303.590000,282.360000,0.038621,1.0,0.408482
...,...,...,...,...,...,...
2082,2024-05-12,2951.000000,2931.000000,0.010926,1.0,0.354857
2083,2024-05-13,2984.551884,2948.303455,-0.009737,0.0,0.463068
2084,2024-05-14,2957.396066,2881.795992,0.054264,1.0,0.432901
2085,2024-05-15,3040.591286,3035.758955,-0.016662,0.0,0.396109


In [11]:
# make sure some of the column types are what hopsworks wants
predictions_updated['date'] = pd.to_datetime(predictions_updated['date']).dt.tz_localize(None)
predictions_updated['tmw_1_0_percent_increase_binary'] = predictions_updated['tmw_1_0_percent_increase_binary'].astype('Int64')

upload_data(predictions_updated, 'eth_ohlc_predictions', 2)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 2087/2087 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: eth_ohlc_predictions_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/711829/jobs/named/eth_ohlc_predictions_2_offline_fg_materialization/executions
